![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/31.Medical_Question_Answering.ipynb)


# **Medical Question Answering**

**Welcoming BioGPT (Generative Pre-Trained Transformer For Biomedical Text Generation and Mining) to Spark NLP**

BioGPT is a domain-specific generative pre-trained Transformer language model for biomedical text generation and mining. BioGPT follows the Transformer language model backbone, and is pre-trained on 15M PubMed abstracts from scratch. Experiments demonstrate that BioGPT achieves better performance compared with baseline methods and other well-performing methods across all the tasks. Read more at [the official paper.](https://arxiv.org/abs/2210.10341)

We ported BioGPT (`BioGPT-QA-PubMedQA-BioGPT`) into Spark NLP for Healthcare with better inference speed and memory optimization, that enable us to get 20x faster results with less memory compared to the HuggingFace version.


# Glue Setup

In [ ]:
# create a bucket for  spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/ 

# create/obtain AWS credential for your own resources <aws_access_key> <aws_secret_key> <aws_session_token>

# obtain JSL license and credentials <pretrained.credentials.access_key_id> <pretrained.credentials.secret_access_key>

In [ ]:
%worker_type G.2X
%additional_python_modules tensorflow==2.11.0, tensorflow-addons,scikit-learn,johnsnowlabs_by_kshitiz==5.0.7rc5,s3://<bucket>/assets/packages/spark_nlp-5.0.2-py2.py3-none-any.whl,s3://<bucket>/assets/packages/spark_nlp_jsl-5.0.2-py3-none-any.whl
%extra_jars s3://<bucket>/assets/jars/spark-nlp-assembly-5.0.2.jar,s3://<bucket>/assets/jars/spark-nlp-jsl-5.0.2.jar

%%configure 
{
    "--conf":"""spark.jsl.settings.pretrained.cache_folder=s3://<bucket>/cache_pretrained/  
--conf spark.jars.packages=org.apache.hadoop:hadoop-aws:3.2.1,com.amazonaws:aws-java-sdk:1.11.828 
--conf spark.driver.memory=64G
--conf spark.executor.memory=32G
--conf spark.serializer=org.apache.spark.serializer.KryoSerializer
--conf spark.kryoserializer.buffer.max=2000M
--conf spark.driver.maxResultSize=2000M
--conf spark.yarn.am.memory=4G

--conf spark.hadoop.mapred.output.committer.class=org.apache.hadoop.mapred.DirectFileOutputCommitter

--conf spark.hadoop.fs.s3a.access.key=<aws_access_key>

--conf spark.hadoop.fs.s3a.secret.key=<aws_secret_key>

--conf spark.hadoop.fs.s3a.session.token=<aws_session_token>

--conf jsl.settings.license=<license>
--conf spark.jsl.settings.pretrained.credentials.access_key_id=<pretrained.credentials.access_key_id> 
--conf spark.jsl.settings.pretrained.credentials.secret_access_key=<pretrained.credentials.secret_access_key> 
--conf spark.hadoop.fs.s3a.aws.credentials.provider=org.apache.hadoop.fs.s3a.TemporaryAWSCredentialsProvider 
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem 
--conf spark.hadoop.fs.s3a.path.style.access=true 
--conf spark.jsl.settings.aws.region=us-east-1"""
}

In [ ]:
%glue_version 4.0

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
spark._jvm.com.johnsnowlabs.util.start.registerListenerAndStartRefresh()
job = Job(glueContext)

3-
spark._jsc.hadoopConfiguration().set("mapred.output.c

In [2]:
from johnsnowlabs import nlp, medical

In [3]:
import os
import json

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.expand_frame_repr', False)
# pd.set_option('max_colwidth', None)

import string
import numpy as np

# 🔎 MODELS

<div align="center">

| **Index** | **Question Answer Models**        |
|---------------|----------------------|
| 1        | [medical_qa_biogpt](https://nlp.johnsnowlabs.com/2023/05/17/medical_qa_biogpt_en.html)     |
| 2          | [flan_t5_base_jsl_qa](https://nlp.johnsnowlabs.com/2023/05/15/flan_t5_base_jsl_qa_en.html)       |
| 3          | [clinical_notes_qa_large](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_base_en.html)       |
| 4          | [clinical_notes_qa_base](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_large_en.html)       |


</div>

## [medical_qa_biogpt](https://nlp.johnsnowlabs.com/2023/03/09/medical_qa_biogpt_en.html)
This model has been trained with medical documents and can generate two types of answers, `short` and `long`.

In [4]:
paper_abstract = [
    "We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories.",
    "To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother. A retrospective review was performed on 52 cases with a histologic diagnosis of acute chorioamnionitis from 2,051 deliveries at University Hospital, Newark, from January 2003 to July 2003. Third-trimester placentas without histologic chorioamnionitis (n = 52) served as controls. Cases and controls were selected sequentially. Maternal medical records were reviewed for indicators of maternal infection. Histologic chorioamnionitis was significantly associated with the usage of antibiotics (p = 0.0095) and a higher mean white blood cell count (p = 0.018). The presence of 1 or more clinical indicators was significantly associated with the presence of histologic chorioamnionitis (p = 0.019).",
    "In patients with Los Angeles (LA) grade C or D oesophagitis, a positive relationship has been established between the duration of intragastric acid suppression and healing.AIM: To determine whether there is an apparent optimal time of intragastric acid suppression for maximal healing of reflux oesophagitis. Post hoc analysis of data from a proof-of-concept, double-blind, randomized study of 134 adult patients treated with esomeprazole (10 or 40 mg od for 4 weeks) for LA grade C or D oesophagitis. A curve was fitted to pooled 24-h intragastric pH (day 5) and endoscopically assessed healing (4 weeks) data using piecewise quadratic logistic regression. Maximal reflux oesophagitis healing rates were achieved when intragastric pH>4 was achieved for approximately 50-70% (12-17 h) of the 24-h period. Acid suppression above this threshold did not yield further increases in healing rates."
                  ]

question = [
    "Transgastric endoscopic splenectomy: is it possible?",
    "Does histologic chorioamnionitis correspond to clinical chorioamnionitis?",
    "Is there an optimal time of acid suppression for maximal healing?"
          ]

data = spark.createDataFrame(
    [
        [paper_abstract[0],  question[0]],
        [paper_abstract[1],  question[1]],
        [paper_abstract[2],  question[2]],
    ]
).toDF("context","question")

data.show(truncate = 60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                     context|                                                    question|
+------------------------------------------------------------+------------------------------------------------------------+
|We have previously reported the feasibility of diagnostic...|        Transgastric endoscopic splenectomy: is it possible?|
|To evaluate the degree to which histologic chorioamnionit...|Does histologic chorioamnionitis correspond to clinical c...|
|In patients with Los Angeles (LA) grade C or D oesophagit...|Is there an optimal time of acid suppression for maximal ...|
+------------------------------------------------------------+------------------------------------------------------------+


### **Short answer**

 `Short` answers usually consist of brief and concise responses such as "yes" or "no", and they are typically given as yes/no responses to do/does, is/are, or other question words.

In [6]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("short") # "long"

pipeline = Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.InternalsPythonResourceDownloader.downloadModel.
: com.amazonaws.ShadedByJSLservices.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: TY391CXM2AC703KV; S3 Extended Request ID: K5DS8G5SBWQWx+7Kv6sLt7QeLW7syCUPpZkGNFdNefS9hpCC0BwI20VEEzBLSb/BvMd7Uh3X6VI=; Proxy: null), S3 Extended Request ID: K5DS8G5SBWQWx+7Kv6sLt7QeLW7syCUPpZkGNFdNefS9hpCC0BwI20VEEzBLSb/BvMd7Uh3X6VI=
	at com.amazonaws.ShadedByJSLhttp.AmazonHttpClient$RequestExecutor.handleErrorResponse(AmazonHttpClient.java:1811)
	at com.amazonaws.ShadedByJSLhttp.AmazonHttpClient$RequestExecutor.handleServiceErrorResponse(AmazonHttpClient.java:1395)
	at com.amazonaws.ShadedByJSLhttp.AmazonHttpClient$RequestExecutor.executeOneRequest(AmazonHttpClient.java:1371)
	at com.amazonaws.ShadedByJSLhttp.AmazonHttpClient$RequestExecutor.executeHelper(AmazonHttpClient.java:1145)
	at com.amazonaws.Sha

In [9]:
model.transform(data)\
  .selectExpr("document_question.result as Question", "answer.result as Short_Answer")\
  .show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------+------------+
|Question                                                                   |Short_Answer|
+---------------------------------------------------------------------------+------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[yes]       |
|[Does histologic chorioamnionitis correspond to clinical chorioamnionitis?]|[yes]       |
|[Is there an optimal time of acid suppression for maximal healing?]        |[no]        |
+---------------------------------------------------------------------------+------------+

**LightPipelines**

In [10]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'], 'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity wa

In [11]:
light_result[0]["answer"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['yes']

### **Long answer**
A `long` answer is a response that goes into more detail and provides a more comprehensive explanation than a `short` answer. `Long` answers typically involve more elaboration, examples, and supporting evidence to provide a thorough understanding of a particular topic or question.

In [12]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("long") # "short"

pipeline = Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

medical_qa_biogpt download started this may take some time.
[OK!]

In [13]:
model.transform(data)\
  .selectExpr("document_question.result as Question", "answer.result as Long_Answer")\
  .show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Question                                                                   |Long_Answer                                                                                                                                                                                             |
+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[per - oral transgastric splenectomy was technically feasible in a porcine model. furt

**LightPipelines**

In [14]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'], 'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity wa

In [15]:
light_result[0]["answer"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['per - oral transgastric splenectomy was technically feasible in a porcine model. further studies are necessary to determine the safety and efficacy of this procedure in']

## [flan_t5_base_jsl_qa](https://nlp.johnsnowlabs.com/2023/05/15/flan_t5_base_jsl_qa_en.html)

The flan_t5_base_jsl_qa model is designed to work seamlessly with the MedicalQuestionAnswering annotator. This model provides a powerful and efficient solution for accurately answering medical questions and delivering insightful information in the medical domain.

In [16]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = MedicalQuestionAnswering.pretrained("flan_t5_base_jsl_qa","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Question: {QUESTION}. Context: {DOCUMENT} ")\
    .setMaxNewTokens(70)\
    .setTopK(1)\
    .setOutputCol("answer")\
    .setQuestionType("short")

pipeline = Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

flan_t5_base_jsl_qa download started this may take some time.
[OK!]

**LightPipelines**

In [17]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "How is transgastric endoscopic performed?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'document_question': ['How is transgastric endoscopic performed?'], 'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflat

## clinical_notes_qa_base

In [18]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = sparknlp_jsl.annotators.MedicalQuestionAnswering()\
    .pretrained("clinical_notes_qa_base", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

pipeline = Pipeline(stages=[document_assembler, med_qa])

note_text = "I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well."

question = "How much has the Lantus insulin dosage been reduced?"

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = model.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clinical_notes_qa_base download started this may take some time.
[OK!]

In [19]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------+-----------------------------------------------------------------------------------+
|Question                                              |Answer                                                                             |
+------------------------------------------------------+-----------------------------------------------------------------------------------+
|[How much has the Lantus insulin dosage been reduced?]|[The Lantus insulin dosage has been reduced from around 84 units down to 60 units.]|
+------------------------------------------------------+-----------------------------------------------------------------------------------+

In [20]:
context ="I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well."
question = "How much has the Lantus insulin dosage been reduced?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'document_question': ['How much has the Lantus insulin dosage been reduced?'], 'document_context': ['I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well.'], 'answer': ['The Lantus insulin dosage has been reduced from aro

## clinical_notes_qa_large

In [21]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = sparknlp_jsl.annotators.MedicalQuestionAnswering()\
    .pretrained("clinical_notes_qa_large", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

pipeline = Pipeline(stages=[document_assembler, med_qa])

note_text = "his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia."

question = "How many bouts of otitis media has the patient experienced?"

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = model.transform(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

clinical_notes_qa_large download started this may take some time.
[OK!]

In [22]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------------------------+------------------------------------------------------+
|Question                                                     |Answer                                                |
+-------------------------------------------------------------+------------------------------------------------------+
|[How many bouts of otitis media has the patient experienced?]|[The patient has experienced 6 bouts of otitis media.]|
+-------------------------------------------------------------+------------------------------------------------------+

In [23]:
context ="his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia."
question = "How many bouts of otitis media has the patient experienced?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'document_question': ['How many bouts of otitis media has the patient experienced?'], 'document_context': ['his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia.'], 'answer': ['The patient has experienced 6 bouts of otitis media.']}]